<a href="https://colab.research.google.com/github/schnap3380/kewwords_for_journal_analysis/blob/main/papers_in_journals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import requests
import time

In [56]:
# Semantic Scholar API key (by request - mine allows for 100 requests / second)
headers={'x-api-key':'my_api_key'}
headers.get('x-api-key')

'FCndxzhW160dParwnevD46jKxnLLuBv7DE3UR1qa'

In [ ]:
df2

In [44]:
def fetch_articles_from_journal(api_key, journal_name):
    url = "https://api.semanticscholar.org/graph/v1/paper/search/bulk"
    offset = 0
    all_articles = []

    params = {
      "venue": journal_name,
      "fields": "title,abstract,year",
      "limit": 1000,
      "apiKey": api_key
    }


    response = requests.get(url, params=params)

    while response.status_code == 429:
      print("Waiting 1 second for access to the API...")
      time.sleep(1)
      response = requests.get(url, params=params)
    if response.status_code != 200:
      raise Exception(f"Error fetching data from Semantic Scholar: {response.status_code}")


    data = response.json()

    if 'data' not in data:
        return pd.DataFrame()  # Return an empty DataFrame if no data is found

  # Process each article and add to the list
    articles = [{
        'Title': article.get('title', ''),
        'Abstract': article.get('abstract', ''),
        'Year': article.get('year', '')
    } for article in data.get('data', [])]

    return pd.DataFrame(articles)

# Example usage:
api_key = 'my_api_key'
journal_name = 'Nature'
df2 = fetch_articles_from_journal(headers, journal_name)

In [42]:
print(df2)

                                                 Title Abstract    Year
0    Induction of proto-oncogene JUN/AP-1 by serum ...     None  1988.0
1    Rotational frequency splitting of solar oscill...     None  1984.0
2                  Nature redesign: what is to dislike     None  2010.0
3    Low-frequency fluctuations of the electric fie...     None  1985.0
4    A dynamic magnetic tension force as the cause ...     None  2015.0
..                                                 ...      ...     ...
995            Holocene climate: Restless carbon pools     None  1999.0
996                               Genetic machinations     None  1987.0
997         Flying Boats for North Atlantic Air-Routes     None  1936.0
998                      The Umdhlebe Tree of Zululand     None     NaN
999  Pulmonary Tumours in Mice induced by Oral Ison...     None  1962.0

[1000 rows x 3 columns]


In [59]:
import pandas as pd
import requests
import time

def fetch_articles_from_journal(api_key, journal_name):
    url = "https://api.semanticscholar.org/graph/v1/paper/search/bulk"
    params = {
        "venue": journal_name,
        "fields": "title,abstract,year",
        "limit": 1000,
        "apiKey": api_key
    }
    all_articles = []

    while True:
        response = requests.get(url, params=params)
        while response.status_code == 429:
            print("Rate limit exceeded, waiting 1 second...")
            time.sleep(1)
            response = requests.get(url, params=params)

        if response.status_code != 200:
            raise Exception(f"API request failed with status {response.status_code}: {response.text}")

        data = response.json()
        articles = data.get('data', [])
        if not articles:
            break

        for article in articles:
            all_articles.append({
                'Title': article.get('title', ''),
                'Abstract': article.get('abstract', ''),
                'Year': article.get('year', '')
            })

        # Check for the presence of a next token in the API response
        if 'next' in data:
            params['cursor'] = data['next']  # Assuming 'next' returns a cursor token for the next page
        else:
            break

    return pd.DataFrame(all_articles)

# Example usage:
api_key = 'my_api_key'
journal_name = 'Mind'
df = fetch_articles_from_journal(api_key, journal_name)
df

{'total': 540, 'token': None, 'data': [{'paperId': '007b41a923c5212520312aa8de61b1c1d1d822a5', 'title': 'Enhancing and Classifying Traffic Signs Using Computer Vision and Deep Convolutional Neural Network', 'abstract': None, 'year': 2020}, {'paperId': '00bff635fe493bfa6bfbf6e82c4efce67e2995b4', 'title': 'OUP accepted manuscript', 'abstract': None, 'year': 2022}, {'paperId': '01172fe80941dbb1e1fd50e68a7ef3b46ca81d04', 'title': 'Rank Offence: The Ecological Theory of Resentment', 'abstract': '\n I argue that fitting resentment tracks unacceptable ‘ecological’ imbalances in relative social strength between victims and perpetrators that arise from violations of legitimate moral expectations. It does not respond purely, or even primarily, to offenders’ attitudes, and its proper targets need not be fully developed moral agents. It characteristically involves a wish for the restoration of social equilibrium rather than a demand for moral recognition or good will. To illuminate these contentio

,Title,Abstract,Year
0,Enhancing and Classifying Traffic Signs Using ...,None,2020
1,OUP accepted manuscript,None,2022
2,Rank Offence: The Ecological Theory of Resentment,\n I argue that fitting resentment tracks unac...,2021
3,Knowability Relative to Information,\n We present a formal semantics for epistemic...,2018
4,Living Toward Virtue: Practical Ethics in the ...,None,2024
...,...,...,...
535,"Machine Learning, Image Processing, Network Se...",None,2020
536,OUP accepted manuscript,None,2021
537,"Conceptual Decolonization, Conceptual Justice,...",\n Calls for decolonization are on the rise in...,2024
538,On the Possibility of Hallucinations,\n Many take the possibility of hallucinations...,2020


In [66]:
df['Abstract'].isna().sum()

408

In [58]:
def fetch_articles_from_journal(api_key, journal_name):
    url = "https://api.semanticscholar.org/graph/v1/paper/search/bulk"
    offset = 0
    all_articles = []

    params = {
      "venue": journal_name,
      "fields": "title,abstract,year",
      "offset": offset,
      "limit": 1000,
      "apiKey": api_key
    }
    #http = requests.get("https://api.semanticscholar.org/graph/v1/paper/search/bulk/%s/papers?limit=1000" %authorId, headers=headers)


    response = requests.get(url, params=params)

    while response.status_code == 429:
      print("Waiting 1 second for access to the API...")
      time.sleep(1)
      response = requests.get(url, params=params)
    if response.status_code != 200:
      raise Exception(f"Error fetching data from Semantic Scholar: {response.status_code}")


    data = response.json()

    if 'data' not in data:
        return pd.DataFrame()  # Return an empty DataFrame if no data is found

  # Process each article and add to the list
    for article in data['data']:
        all_articles.append({
            'Title': article.get('title', ''),
            'Abstract': article.get('abstract', ''),
            'Year': article.get('year', '')
        })

  # Update offset to fetch next set of articles
    offset += 100  # Increment offset by the limit
    return pd.DataFrame(all_articles)

# Example usage:
api_key = 'my_api_key'
journal_name = 'Mind'
df2 = fetch_articles_from_journal(api_key, journal_name)
df2

,Title,Abstract,Year
0,Enhancing and Classifying Traffic Signs Using ...,None,2020
1,OUP accepted manuscript,None,2022
2,Rank Offence: The Ecological Theory of Resentment,\n I argue that fitting resentment tracks unac...,2021
3,Knowability Relative to Information,\n We present a formal semantics for epistemic...,2018
4,Living Toward Virtue: Practical Ethics in the ...,None,2024
...,...,...,...
535,"Machine Learning, Image Processing, Network Se...",None,2020
536,OUP accepted manuscript,None,2021
537,"Conceptual Decolonization, Conceptual Justice,...",\n Calls for decolonization are on the rise in...,2024
538,On the Possibility of Hallucinations,\n Many take the possibility of hallucinations...,2020
